## 데이터 전처리
데이터 분석에 앞서, 분석에 필요한 데이터들로 전처리 작업을 해줍니다.

### 패키지 설치

In [19]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re

### 시즌아이디(seasonId) 메타데이터 조회
필요한 데이터는 챔피언스리그 시즌 데이터 이므로 챔피언스리그 시즌에 대응하는 아이디 값만 필터링 해줍니다.

In [11]:
seasonId_res = requests.get('https://static.api.nexon.co.kr/fifaonline4/latest/seasonid.json')

if seasonId_res.status_code == 200:
    seasonId_parsed_data = seasonId_res.json()
    seasonId_data = pd.DataFrame(seasonId_parsed_data)
elif seasonId_res.status_code == 404:
    print('Not Found.')
else:
    print('An error has occurred.')

uefa_data = seasonId_data.loc[seasonId_data['className'].str.contains('UEFA')]

In [12]:
uefa_data.head()

,seasonId,className,seasonImg
16,221,19 UCL (19 UEFA Champions League),https://ssl.nexon.com/s2/game/fo4/obt/external...
29,242,20 UCL (20 UEFA Champions League),https://ssl.nexon.com/s2/game/fo4/obt/external...
42,260,21 UCL (21 UEFA Champions League),https://ssl.nexon.com/s2/game/fo4/obt/external...


챔피언스리그 시즌 아이디는 221, 242, 260 임을 알 수 있습니다.

### 선수 고유 식별자(spid) 메타데이터 조회
선수 고유 식별자는 시즌아이디 (seasonid) 3자리 + 선수아이디 (pid) 6자리로 구성되어 있습니다.
앞서 필터링한 챔피언스리그 시즌 아이디를 사용하여 챔피언스리그에 출전한 선수들로만 추가 필터링을 해줍니다.

In [13]:
spId_res = requests.get('https://static.api.nexon.co.kr/fifaonline4/latest/spid.json')

if spId_res.status_code == 200:
    spId_parsed_data = spId_res.json()
    spId_data = pd.DataFrame(spId_parsed_data)
elif spId_res.status_code == 404:
    print('Not Found.')
else:
    print('An error has occurred.')

uefa19_data = spId_data.loc[spId_data['id'].astype(str).str.startswith('221')]
uefa20_data = spId_data.loc[spId_data['id'].astype(str).str.startswith('242')]
uefa21_data = spId_data.loc[spId_data['id'].astype(str).str.startswith('260')]

In [14]:
uefa21_data.head()

,id,name
6819,260002147,M. 스테켈렌뷔르흐
6820,260020801,크리스티아누 호날두
6821,260124375,부라크 일마즈
6822,260135507,페르난지뉴
6823,260138412,제임스 밀너


분석에 필요한 데이터는 첼시 선수들 한정이므로 팀 구분자를 제공해주면 좋겠지만 아쉽게도 제공해주지 않네요.
첼시 선수 명단을 직접 넣어줘야 할 것 같습니다.

In [15]:
chelsea_player = [
    '안토니오 뤼디거',
    '티아구 실바',
    '티모 베르너',
    '에두아르 멘디',
    'A. 크리스텐센',
    '리스 제임스',
    '로멜루 루카쿠',
    '조르지뉴',
    '하킴 지예시',
    'C. 허드슨-오도이',
    '아스필리쿠에타',
    '루벤 로프터스-칙',
    '카이 하베르츠',
    '트레보 찰로바',
    '메이슨 마운트',
    '마르코스 알론소',
    '벤 칠웰',
    '사울',
    '은골로 캉테',
    '크리스천 풀리식',
    '말랑 사르',
    '로스 바클리',
    '케파',
    'M. 베티넬리'
]

uefa19_chelsea_data = uefa19_data.loc[spId_data['name'].str.contains('|'.join(chelsea_player))]
uefa20_chelsea_data = uefa20_data.loc[spId_data['name'].str.contains('|'.join(chelsea_player))]
uefa21_chelsea_data = uefa21_data.loc[spId_data['name'].str.contains('|'.join(chelsea_player))]

In [16]:
uefa21_chelsea_data.head()

,id,name
6833,260164240,티아구 실바
6862,260184432,아스필리쿠에타
6884,260192505,로멜루 루카쿠
6885,260192638,마르코스 알론소
6909,260199189,로스 바클리


### 데이터 크롤링
각 선수들의 데이터 조회는 피파온라인4 데이터센터 사이트의 선수 상세 정보 페이지에서 크롤링 하는 방향으로 진행해야 할 것 같습니다.

https://fifaonline4.nexon.com/DataCenter/PlayerInfo?spid={선수 고유 식별자}

In [17]:
url = 'https://fifaonline4.nexon.com/DataCenter/PlayerInfo?spid='
class_ids = pd.concat([uefa19_chelsea_data['id'], uefa20_chelsea_data['id'], uefa21_chelsea_data['id']])

player_df = pd.DataFrame()

for idx, val in enumerate(class_ids):
    response = requests.get(url + str(val))

    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        header = '#middle .datacenter .player_view .content_header'
        middle = '#middle .datacenter .player_view .content_middle'
        bottom = '#middle .datacenter .player_view .content_bottom'
        name = soup.select_one(header + ' ' + '.info_name .name').getText()
        position = soup.select_one(header + ' ' + '.thumb .position').getText()
        overall = soup.select_one(header + ' ' + '.thumb .ovr').getText()
        birth = soup.select_one(header + ' ' + '.info_etc .birth').getText().strip()
        height = soup.select_one(header + ' ' + '.info_etc .height').getText()
        weight = soup.select_one(header + ' ' + '.info_etc .weight').getText()
        physical = soup.select_one(header + ' ' + '.info_etc .physical').getText()
        skill = soup.select_one(header + ' ' + '.info_etc .skill').getText().strip()
        foot = soup.select_one(header + ' ' + '.info_etc .foot').getText().strip()
        season = soup.select_one(header + ' ' + '.info_etc .season').getText()
        team = soup.select_one(header + ' ' + '.info_team .team .txt').getText()
        nation = soup.select_one(header + ' ' + '.info_team .nation .txt').getText()
        skill_wrap = soup.select_one(header + ' ' + '.skill_wrap').getText()
        overall_stats = soup.select(middle + ' ' + '.txt')
        overall_values = soup.select(middle + ' ' + '.value')
        detail_stats = soup.select(bottom + ' ' + '.txt')
        detail_values = soup.select(bottom + ' ' + '.value')

        if str(val).startswith('221'):
            player_data = pd.DataFrame([{'class': '19'}])
        elif str(val).startswith('242'):
            player_data = pd.DataFrame([{'class': '20'}])
        elif str(val).startswith('260'):
            player_data = pd.DataFrame([{'class': '21'}])
        else:
            player_data = pd.DataFrame([{'class': np.nan}])
        player_data['name'] = name
        player_data['position'] = position
        player_data['overall'] = overall
        player_data['birth'] = birth
        player_data['height'] = height
        player_data['weight'] = weight
        player_data['physical'] = physical
        player_data['skill'] = skill
        player_data['foot'] = foot
        player_data['season'] = season
        player_data['team'] = team
        player_data['nation'] = nation
        player_data['skill_wrap'] = re.sub("\s|특성", "", skill_wrap)
        for s, v in zip(overall_stats, overall_values):
            player_data['오버롤_' + s.getText()] = v.getText()
        for s, v in zip(detail_stats, detail_values):
            player_data[s.getText()] = v.getText()
        player_data['클래스_이름_포지션_오버롤'] = player_data.apply(lambda x: str(x['class']) + '_' + x['name'] + '_' + x['position'] + '_' + str(x['overall']), axis=1)

        player_df = pd.concat([player_df, player_data])



    elif response.status_code == 404:
        print('Not Found.')
    else:
        print('An error has occurred.')

with pd.ExcelWriter('player_data_fifa_online.xlsx') as writer:
    player_df.to_excel(writer, index=False)

In [18]:
player_df.head()

,class,name,position,overall,birth,height,weight,physical,skill,foot,...,GK 핸들링,GK 킥,GK 반응속도,GK 위치선정,클래스_이름_포지션_오버롤,오버롤_다이빙,오버롤_핸들링,오버롤_킥,오버롤_반응속도,오버롤_위치선정
0,19,티아구 실바,CB,96,1984.09.22 (39세),183cm,79kg,보통,★★☆☆☆☆,L3 – R5,...,19,12,17,16,19_티아구 실바_CB_96,NaN,NaN,NaN,NaN,NaN
0,19,아스필리쿠에타,RB,90,1989.08.28 (34세),178cm,77kg,보통,★★☆☆☆☆,L3 – R5,...,9,10,5,6,19_아스필리쿠에타_RB_90,NaN,NaN,NaN,NaN,NaN
0,19,마르코스 알론소,LB,87,1990.12.28 (33세),188cm,85kg,마름,★★★☆☆☆,L5 – R3,...,7,13,12,9,19_마르코스 알론소_LB_87,NaN,NaN,NaN,NaN,NaN
0,19,로스 바클리,CAM,74,1993.12.05 (30세),186cm,87kg,보통,★★★★☆☆,L5 – R5,...,14,12,10,6,19_로스 바클리_CAM_74,NaN,NaN,NaN,NaN,NaN
0,19,안토니오 뤼디거,CB,81,1993.03.03 (30세),190cm,85kg,마름,★★☆☆☆☆,L3 – R5,...,15,10,8,12,19_안토니오 뤼디거_CB_81,NaN,NaN,NaN,NaN,NaN


,class,name,position,overall,birth,height,weight,physical,skill,foot,...,GK 핸들링,GK 킥,GK 반응속도,GK 위치선정,클래스_이름_포지션_오버롤,오버롤_다이빙,오버롤_핸들링,오버롤_킥,오버롤_반응속도,오버롤_위치선정
0,19,티아구 실바,CB,96,1984.09.22 (39세),183cm,79kg,보통,★★☆☆☆☆,L3 – R5,...,19,12,17,16,19_티아구 실바_CB_96,NaN,NaN,NaN,NaN,NaN
0,19,아스필리쿠에타,RB,90,1989.08.28 (34세),178cm,77kg,보통,★★☆☆☆☆,L3 – R5,...,9,10,5,6,19_아스필리쿠에타_RB_90,NaN,NaN,NaN,NaN,NaN
0,19,마르코스 알론소,LB,87,1990.12.28 (33세),188cm,85kg,마름,★★★☆☆☆,L5 – R3,...,7,13,12,9,19_마르코스 알론소_LB_87,NaN,NaN,NaN,NaN,NaN
0,19,로스 바클리,CAM,74,1993.12.05 (30세),186cm,87kg,보통,★★★★☆☆,L5 – R5,...,14,12,10,6,19_로스 바클리_CAM_74,NaN,NaN,NaN,NaN,NaN
0,19,안토니오 뤼디거,CB,81,1993.03.03 (30세),190cm,85kg,마름,★★☆☆☆☆,L3 – R5,...,15,10,8,12,19_안토니오 뤼디거_CB_81,NaN,NaN,NaN,NaN,NaN
